In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import progressbar
import random
import seaborn as sns
import time

import Simulation

from IPython.display import clear_output
from Diffusion.Model       import *
from Algorithms.Heuristics import *
from Algorithms.Baselines  import *

sns.set_style("darkgrid")

In [2]:
algorithms = [random_solution, degree_solution]
n_trials = 50
seed_sizes = list(range(0, 100+1, 10))
seed_sizes[0] = 1
time_horizon = 100
topo_code = "eu-core" #"amazon"

columns = ["trial", "opinion", "activated", "algorithm", "seed_size", "alg-runtime"]
data = {col: [] for col in columns}
random_seed = 16785407

uniform = random.random

# Initial Observations & To-Do's
* First, there is ***something*** wrong with the BIC diffusion model. This is obvious because (using the random solution) when $k=1000$ we get the bizarre case where $\sum_{v\in V}O_v^T < 1000$. This should *not* be happening. **Investigate it.**
* Second, the TIM+ algorithm is not fully implemented. We need to get the set of nodes that have been *visited*. This should be doable with the killed set and active set, but investigate it with the BIC diffusion model.

In [3]:
TIM_start = time.time()
df_TIM = Simulation.run(
    topo_code=topo_code, 
    algorithm=TIM_solution, 
    seed_sizes=seed_sizes, 
    time_horizon=time_horizon, 
    n_trials=n_trials, 
    ffm_distr_func=uniform,
    opinion_distr_func=uniform,
    random_seed=None,#random_seed, 
    use_communities=False,
    out_dir=None
)
TIM_runtime = time.time() - degr_start
print(f"\n<> Simulation using `{TIM_solution.__name__}` took {TIM_runtime:0.5f} seconds.")

0%|          | 0/550 [00:00<?, ?it/s]

AttributeError: 'hypergraph' object has no attribute '_nodes_edgesets'

In [4]:
rand_start = time.time()
df_rand = Simulation.run(
    topo_code=topo_code, 
    algorithm=random_solution, 
    seed_sizes=seed_sizes, 
    time_horizon=time_horizon, 
    n_trials=n_trials, 
    ffm_distr_func=uniform,
    opinion_distr_func=uniform,
    random_seed=None,#random_seed, 
    use_communities=False,
    out_dir=None
)
rand_runtime = time.time() - rand_start
print(f"<> Simulation using `{random_solution.__name__}` took {rand_runtime:0.5f} seconds.")

▊       | 155/550 [00:12<00:47,  8.29it/s]
random_solution, K = 50, Trial (20/50):  49%|████▉     | 269/550 [00:31<00:47,  5.90it/s]


In [ ]:
degr_start = time.time()
df_degr = Simulation.run(
    topo_code=topo_code, 
    algorithm=degree_solution, 
    seed_sizes=seed_sizes, 
    time_horizon=time_horizon, 
    n_trials=n_trials, 
    ffm_distr_func=uniform,
    opinion_distr_func=uniform,
    random_seed=None,#random_seed, 
    use_communities=False,
    out_dir=None
)
degr_runtime = time.time() - degr_start
print(f"\n<> Simulation using `{degree_solution.__name__}` took {degr_runtime:0.5f} seconds.")

In [ ]:
df = df_rand.copy()
df = df.append(df_degr)
df = df.append(df_TIM)
sns.lineplot(x="seed_size", y="opinion", markers="os*", err_style="bars", style="algorithm", hue="algorithm", data=df)
plt.show()

sns.barplot(x="seed_size", y="activated", hue="algorithm", data=df)
plt.show()

In [19]:
df.head()

,trial,opinion,activated,algorithm,seed_size,algorithm time
0,0,167435.780858,1,random_solution,1,0.005466
1,1,167436.022168,1,random_solution,1,0.009532
2,2,167436.989482,2,random_solution,1,0.008146
3,3,167438.472127,2,random_solution,1,0.009428
4,4,167439.459651,1,random_solution,1,0.007761
